In [13]:
import numpy as np
from scipy.sparse import csr_matrix

In [40]:
wv_exp = np.linspace(300, 322, num=5000)
wv_lg = np.linspace(302, 320, num=500)

In [41]:
delta_wv = wv_exp[np.newaxis].transpose() - wv_lg
delta_wv

array([[ -2.        ,  -2.03607214,  -2.07214429, ..., -19.92785571,
        -19.96392786, -20.        ],
       [ -1.99559912,  -2.03167126,  -2.06774341, ..., -19.92345483,
        -19.95952698, -19.99559912],
       [ -1.99119824,  -2.02727038,  -2.06334253, ..., -19.91905395,
        -19.9551261 , -19.99119824],
       ...,
       [ 19.99119824,  19.9551261 ,  19.91905395, ...,   2.06334253,
          2.02727038,   1.99119824],
       [ 19.99559912,  19.95952698,  19.92345483, ...,   2.06774341,
          2.03167126,   1.99559912],
       [ 20.        ,  19.96392786,  19.92785571, ...,   2.07214429,
          2.03607214,   2.        ]])

In [42]:
fwhm = 0.1
threshold = 3
delta_wv[delta_wv<-fwhm*threshold] = 0
delta_wv[delta_wv> fwhm*threshold] = 0
delta_wv

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [43]:
delta_wv = csr_matrix(delta_wv)
delta_wv

<5000x500 sparse matrix of type '<class 'numpy.float64'>'
	with 68164 stored elements in Compressed Sparse Row format>

In [44]:
%timeit delta_wv.expm1()

89.4 µs ± 4.08 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [45]:
delta_wv_array = delta_wv.toarray()
%timeit np.exp(delta_wv_array)

4.86 ms ± 132 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [46]:
%timeit np.exp(delta_wv.data)

31.7 µs ± 2.36 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
